<a href="https://colab.research.google.com/github/isamu-isozaki/toxic-joke-generator/blob/master/GPT_Jokes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Thanks https://www.reddit.com/r/MachineLearning/comments/bgvzdu/d_jokes_generated_via_gpt2/
import os
import json
import random
import re

In [7]:
!pip install -U -q PyDrive

     |████████████████████████████████| 993kB 2.8MB/s 


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
folder_id = None

In [0]:
def ListFolder(parent, show=False):
  global folder_id
  filelist=[]
  file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % parent}).GetList()
  for f in file_list:
    if show:
      print(f"{f['title']}, {f['id']}")
    if f['mimeType']=='application/vnd.google-apps.folder': # if folder
        if f['title'] == "autoencoder1":
          folder_id = f['id']
          filelist.append({"id":f['id'],"title":f['title'], "mimeType": 'application/vnd.google-apps.folder', "list":ListFolder(f['id'])})
        else:
          for f1 in ListFolder(f['id']):
            filelist.append(f1)
    else:
        filelist.append({"id":f['id'],"title":f['title'],"mimeType":"file","title1":f['alternateLink']})
  return filelist
fl = ListFolder('root')

In [12]:
!git clone https://github.com/isamu-isozaki/toxic-joke-generator.git

Cloning into 'toxic-joke-generator'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 106 (delta 3), reused 0 (delta 0), pack-reused 97
Receiving objects: 100% (106/106), 211.17 MiB | 11.27 MiB/s, done.
Resolving deltas: 100% (36/36), done.
Checking out files: 100% (47/47), done.


In [13]:
cd toxic-joke-generator

/content/toxic-joke-generator


In [14]:
!python download_model.py 117M

Fetching checkpoint: 1.00kit [00:00, 777kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 34.7Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.01Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:10, 48.5Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 4.60Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 33.4Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 32.7Mit/s]                                                       


In [0]:
if not os.path.exists('checkpoint'):
  os.makedirs("checkpoint")

In [16]:
cd checkpoint

/content/toxic-joke-generator/checkpoint


In [0]:
if not os.path.exists('run1'):
  os.makedirs("run1")

In [0]:
for f in fl:
  if f['title'] == "autoencoder1":
    for part in f["list"]:
      fil = drive.CreateFile({'id':part['id']})
      fil.GetContentFile(part['title'][18:])
      os.rename(part['title'][18:], "./run1/"+part['title'][18:])

In [19]:
cd ..

/content/toxic-joke-generator


In [20]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 604kB 7.6MB/s 
     |████████████████████████████████| 51kB 19.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
ERROR: spacy 2.0.18 has requirement regex==2018.01.10, but you'll have regex 2017.4.5 which is incompatible.
  Found existing installation: regex 2018.1.10
    Uninstalling regex-2018.1.10:
      Successfully uninstalled regex-2018.1.10
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
!python ./train.py --dataset jokes_400_2_3.0_1.txt.npz --batch_size 2 --sample_every 100 --save_every 1000 --folder_id $folder_id

/usr/local/lib/python3.6/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
2019-06-03 15:39:46.510289: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-03 15:39:46.510643: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x71e9700 executing computations on platform Host. Devices:
2019-06-03 15:39:46.510683: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-06-03 15:39:46.680954: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-06-03 15:39:46.681540: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x71e95a0 executing computations on platform CUDA. Devices:
2019-

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

In [0]:
!python3 generate_unconditional_samples.py --top_k 40 --temperature 1.2